<a href="https://colab.research.google.com/github/sanghyun-ai/ktcloud_genai/blob/main/110_LLM_Tokenization_%E1%84%8B%E1%85%AA%E1%86%AB%E1%84%89%E1%85%A5%E1%86%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tokenization**



---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---



# **Tokenization 이란?**

- 텍스트를 컴퓨터가 처리할 수 있는 의미있는 단위(토큰)로 나누는 전처리 과정
- 기본 아이디어
    - 텍스트 → 토큰 리스트 변환
    - AI 모델이 이해할 수 있는 최소 의미 단위(=토큰)로 분할
    - 언어마다 다른 방식 적용
- 토큰화 방법

|방법|설명|예시|장점|단점|
|---|---|---|---|---|
|단어 단위|공백으로 분할"|I love AI" → ["I", "love", "AI"]|직관적어휘 크기 폭발|
|문자 단위|글자별 분할|"Hello" → ["H", "e", "l", "l", "o"]|어휘 크기 작음|의미 정보 손실|
|서브워드|단어를 더 작은 단위로|"unknown" → ["un", "know", "n"]|균형잡힌 접근|복잡한 알고리즘|
|형태소 분석|의미 단위로 분할|"먹었다" → ["먹", "었", "다"]|언어학적 정확성|언어별 특화 필요|

### 예제 1: 기본 토큰화 방법들

In [1]:
import re
from collections import Counter

def basic_tokenization_examples():
    text = "Hello, World! I'm learning NLP. It's amazing!"

    print("=== 기본 토큰화 방법들 ===")
    print(f"원본 텍스트: {text}")

    # 1. 공백 기반 분할
    whitespace_tokens = text.split()
    print(f"\n1. 공백 분할: {whitespace_tokens}")

    # 2. 정규식으로 단어 추출
    word_tokens = re.findall(r'\w+', text.lower())
    print(f"2. 단어만 추출: {word_tokens}")

    # 3. 문자 단위 분할
    char_tokens = list(text.lower())
    print(f"3. 문자 분할: {char_tokens}")

    # 4. 구두점 포함 분할
    punct_tokens = re.findall(r'\w+|[^\w\s]', text)
    print(f"4. 구두점 포함: {punct_tokens}")

basic_tokenization_examples()

=== 기본 토큰화 방법들 ===
원본 텍스트: Hello, World! I'm learning NLP. It's amazing!

1. 공백 분할: ['Hello,', 'World!', "I'm", 'learning', 'NLP.', "It's", 'amazing!']
2. 단어만 추출: ['hello', 'world', 'i', 'm', 'learning', 'nlp', 'it', 's', 'amazing']
3. 문자 분할: ['h', 'e', 'l', 'l', 'o', ',', ' ', 'w', 'o', 'r', 'l', 'd', '!', ' ', 'i', "'", 'm', ' ', 'l', 'e', 'a', 'r', 'n', 'i', 'n', 'g', ' ', 'n', 'l', 'p', '.', ' ', 'i', 't', "'", 's', ' ', 'a', 'm', 'a', 'z', 'i', 'n', 'g', '!']
4. 구두점 포함: ['Hello', ',', 'World', '!', 'I', "'", 'm', 'learning', 'NLP', '.', 'It', "'", 's', 'amazing', '!']


### 예제 2: 한국어 토큰화

In [2]:
def korean_tokenization():
    korean_text = "안녕하세요! 저는 한국어를 공부하고 있습니다."

    print("\n=== 한국어 토큰화 ===")
    print(f"원본 텍스트: {korean_text}")

    # 1. 공백 기반 (한국어는 부정확)
    space_tokens = korean_text.split()
    print(f"1. 공백 분할: {space_tokens}")

    # 2. 문자 단위
    char_tokens = list(korean_text)
    print(f"2. 문자 분할: {char_tokens}")

    # 3. 간단한 한국어 토큰화 (실제로는 형태소 분석기 사용)
    # 여기서는 어절 단위로 분리
    simple_tokens = re.findall(r'[가-힣]+|[a-zA-Z]+|[!?.]', korean_text)
    print(f"3. 간단한 분할: {simple_tokens}")

korean_tokenization()


=== 한국어 토큰화 ===
원본 텍스트: 안녕하세요! 저는 한국어를 공부하고 있습니다.
1. 공백 분할: ['안녕하세요!', '저는', '한국어를', '공부하고', '있습니다.']
2. 문자 분할: ['안', '녕', '하', '세', '요', '!', ' ', '저', '는', ' ', '한', '국', '어', '를', ' ', '공', '부', '하', '고', ' ', '있', '습', '니', '다', '.']
3. 간단한 분할: ['안녕하세요', '!', '저는', '한국어를', '공부하고', '있습니다', '.']


### 예제 3: 서브워드 토큰화 시뮬레이션

In [3]:
def subword_tokenization_demo():
    words = ["unknown", "unhappy", "replay", "preprocessing"]

    print("\n=== 서브워드 토큰화 데모 ===")

    # 간단한 BPE 시뮬레이션
    vocab = set()

    # 모든 문자를 기본 어휘에 추가
    for word in words:
        vocab.update(list(word))

    # 자주 나오는 문자 조합 찾기
    bigrams = Counter()
    for word in words:
        for i in range(len(word) - 1):
            bigrams[word[i:i+2]] += 1

    # 가장 빈번한 바이그램을 어휘에 추가
    common_bigrams = bigrams.most_common(5)
    for bigram, count in common_bigrams:
        vocab.add(bigram)
        print(f"추가된 서브워드: '{bigram}' (빈도: {count})")

    print(f"\n최종 어휘: {sorted(vocab)}")

    # 단어를 서브워드로 분해 (단순화된 버전)
    def tokenize_word(word):
        tokens = []
        i = 0
        while i < len(word):
            # 가장 긴 매칭 찾기
            found = False
            for length in range(min(3, len(word) - i), 0, -1):
                subword = word[i:i+length]
                if subword in vocab:
                    tokens.append(subword)
                    i += length
                    found = True
                    break
            if not found:
                tokens.append(word[i])
                i += 1
        return tokens

    print(f"\n=== 서브워드 토큰화 결과 ===")
    for word in words:
        tokens = tokenize_word(word)
        print(f"{word:15} → {tokens}")

subword_tokenization_demo()


=== 서브워드 토큰화 데모 ===
추가된 서브워드: 'un' (빈도: 2)
추가된 서브워드: 're' (빈도: 2)
추가된 서브워드: 'ep' (빈도: 2)
추가된 서브워드: 'pr' (빈도: 2)
추가된 서브워드: 'nk' (빈도: 1)

최종 어휘: ['a', 'c', 'e', 'ep', 'g', 'h', 'i', 'k', 'l', 'n', 'nk', 'o', 'p', 'pr', 'r', 're', 's', 'u', 'un', 'w', 'y']

=== 서브워드 토큰화 결과 ===
unknown         → ['un', 'k', 'n', 'o', 'w', 'n']
unhappy         → ['un', 'h', 'a', 'p', 'p', 'y']
replay          → ['re', 'p', 'l', 'a', 'y']
preprocessing   → ['pr', 'ep', 'r', 'o', 'c', 'e', 's', 's', 'i', 'n', 'g']


### 예제 4: 실제 AI 모델에서 사용하는 토큰화

In [4]:
def ai_model_tokenization():
    sentences = [
        "I love artificial intelligence!",
        "Machine learning is fascinating.",
        "Natural language processing rocks!"
    ]

    print("\n=== AI 모델용 토큰화 ===")

    # 1. 어휘 사전 구축
    word_counter = Counter()
    for sentence in sentences:
        words = re.findall(r'\w+', sentence.lower())
        word_counter.update(words)

    # 2. 특수 토큰 추가
    vocab = {
        '<pad>': 0,    # 패딩
        '<unk>': 1,    # 미지 단어
        '<start>': 2,  # 시작
        '<end>': 3     # 끝
    }

    # 3. 빈번한 단어들 추가
    for word, count in word_counter.most_common():
        if count >= 1:  # 최소 빈도
            vocab[word] = len(vocab)

    print(f"어휘 사전 크기: {len(vocab)}")
    print(f"어휘: {list(vocab.keys())}")

    # 4. 문장을 토큰 ID로 변환
    def sentence_to_ids(sentence, vocab, max_len=10):
        words = re.findall(r'\w+', sentence.lower())
        ids = [vocab['<start>']]

        for word in words:
            if len(ids) >= max_len - 1:  # <end> 공간 확보
                break
            ids.append(vocab.get(word, vocab['<unk>']))

        ids.append(vocab['<end>'])

        # 패딩
        while len(ids) < max_len:
            ids.append(vocab['<pad>'])

        return ids[:max_len]

    print(f"\n=== 토큰 ID 변환 ===")
    for sentence in sentences:
        token_ids = sentence_to_ids(sentence, vocab)
        print(f"'{sentence}'")
        print(f"토큰 ID: {token_ids}")

        # 역변환으로 확인
        id_to_vocab = {v: k for k, v in vocab.items()}
        recovered_tokens = [id_to_vocab[id] for id in token_ids]
        print(f"토큰: {recovered_tokens}")
        print()

ai_model_tokenization()


=== AI 모델용 토큰화 ===
어휘 사전 크기: 16
어휘: ['<pad>', '<unk>', '<start>', '<end>', 'i', 'love', 'artificial', 'intelligence', 'machine', 'learning', 'is', 'fascinating', 'natural', 'language', 'processing', 'rocks']

=== 토큰 ID 변환 ===
'I love artificial intelligence!'
토큰 ID: [2, 4, 5, 6, 7, 3, 0, 0, 0, 0]
토큰: ['<start>', 'i', 'love', 'artificial', 'intelligence', '<end>', '<pad>', '<pad>', '<pad>', '<pad>']

'Machine learning is fascinating.'
토큰 ID: [2, 8, 9, 10, 11, 3, 0, 0, 0, 0]
토큰: ['<start>', 'machine', 'learning', 'is', 'fascinating', '<end>', '<pad>', '<pad>', '<pad>', '<pad>']

'Natural language processing rocks!'
토큰 ID: [2, 12, 13, 14, 15, 3, 0, 0, 0, 0]
토큰: ['<start>', 'natural', 'language', 'processing', 'rocks', '<end>', '<pad>', '<pad>', '<pad>', '<pad>']




---



# 현대 AI에서 사용하는 토크나이저


|토크나이저|사용 모델|특징
|---|---|---|
|BPE|GPT 시리즈|바이트 쌍 인코딩|
|SentencePiece|T5, mT5|언어 무관한 서브워드|
|WordPiece|BERT|구글 개발, 한국어 지원|

- 핵심 포인트
    - 전처리의 첫 단계: 모든 NLP 태스크의 시작점
    - 언어별 최적화: 한국어는 형태소 분석 필수
    - 균형점 찾기: 의미 vs 어휘 크기 vs 계산 효율성
    - 특수 토큰: <pad>, <unk>, <start>, <end> 등으로 모델 제어

- 비유:
    - "문장을 레고 블록으로 분해하는 것" - 컴퓨터가 조립할 수 있는 크기로!
    - 토큰화는 모든 NLP 모델의 입구이며, 성능에 큰 영향을 미치는 중요한 전처리 과정이다!



---



# Transformers 토크나이저 사용 예제

### **예제 1: 토크나이제이션 과정 단계별 확인**

In [5]:
# 경고 메시지만 숨기기
import warnings
import os

# 1. 경고 메시지 비활성화
warnings.filterwarnings("ignore", category=UserWarning)

# 2. Hugging Face 진행률 표시 끄기
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

In [6]:
from transformers import GPT2Tokenizer
import torch

# 토크나이저 로드
tokenizer = GPT2Tokenizer.from_pretrained('gpt2') # 무료 공개 모델

# 원본 텍스트
input_text = "프로그래밍은 재미있다. 프로그래밍은"
print(f"✅ 원본 텍스트: '{input_text}'")
print()

# 1단계: 토큰으로 분할 (문자열 형태)
tokens = tokenizer.tokenize(input_text)
print(f"1️⃣ 토큰 분할 결과: {tokens}")
print()

# 2단계: 각 토큰의 ID 확인
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"2️⃣ 토큰 ID들: {token_ids}")
print()

# 3단계: encode 함수로 한번에 처리
input_ids_list = tokenizer.encode(input_text)
print(f"3️⃣ encode 결과 (리스트): {input_ids_list}")
print()

# 4단계: 텐서 형태로 변환
input_ids = tokenizer.encode(input_text, return_tensors='pt')
print(f"4️⃣ 텐서 형태로 변환")
print(f"⭢ input_ids (텐서): {input_ids}")
print(f"⭢ input_ids 형태: {input_ids.shape}")
print(f"⭢ input_ids 타입: {type(input_ids)}")
print()

# 역변환: ID를 다시 텍스트로
decoded_text = tokenizer.decode(input_ids[0])
print(f"✅ 역변환 결과: '{decoded_text}'")


✅ 원본 텍스트: '프로그래밍은 재미있다. 프로그래밍은'

1️⃣ 토큰 분할 결과: ['í', 'Ķ', 'Ħ', 'ë', '¡', 'ľ', 'ê', '·', '¸', 'ë', 'ŀ', 'ĺ', 'ë', '°', 'į', 'ìĿ', 'Ģ', 'Ġì', 'ŀ', '¬', 'ë', '¯', '¸', 'ì', 'ŀ', 'Ī', 'ëĭ', '¤', '.', 'Ġ', 'í', 'Ķ', 'Ħ', 'ë', '¡', 'ľ', 'ê', '·', '¸', 'ë', 'ŀ', 'ĺ', 'ë', '°', 'į', 'ìĿ', 'Ģ']

2️⃣ 토큰 ID들: [169, 242, 226, 167, 94, 250, 166, 115, 116, 167, 252, 246, 167, 108, 235, 35975, 222, 23821, 252, 105, 167, 107, 116, 168, 252, 230, 46695, 97, 13, 220, 169, 242, 226, 167, 94, 250, 166, 115, 116, 167, 252, 246, 167, 108, 235, 35975, 222]

3️⃣ encode 결과 (리스트): [169, 242, 226, 167, 94, 250, 166, 115, 116, 167, 252, 246, 167, 108, 235, 35975, 222, 23821, 252, 105, 167, 107, 116, 168, 252, 230, 46695, 97, 13, 220, 169, 242, 226, 167, 94, 250, 166, 115, 116, 167, 252, 246, 167, 108, 235, 35975, 222]

4️⃣ 텐서 형태로 변환
⭢ input_ids (텐서): tensor([[  169,   242,   226,   167,    94,   250,   166,   115,   116,   167,
           252,   246,   167,   108,   235, 35975,   222, 23821,   252,   105,
       

## **예제 2: 다양한 텍스트의 토크나이제이션 비교**

In [7]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 다양한 텍스트 예제
texts = [
    "안녕하세요",
    "Hello world",
    "프로그래밍",
    "AI는 미래다",
    "123456",
    "hello@email.com"
]

print("=== 다양한 텍스트의 토크나이제이션 결과 ===")
for text in texts:
    tokens = tokenizer.tokenize(text)
    input_ids = tokenizer.encode(text, return_tensors='pt')

    print(f"텍스트: '{text}'")
    print(f"토큰: {tokens}")
    print(f"input_ids: {input_ids.tolist()}")
    print(f"토큰 개수: {len(tokens)}")
    print("-" * 50)

=== 다양한 텍스트의 토크나이제이션 결과 ===
텍스트: '안녕하세요'
토큰: ['ì', 'ķ', 'Ī', 'ë', 'ħ', 'ķ', 'íķ', 'ĺ', 'ì', 'Ħ', '¸', 'ì', 'ļ', 'Ķ']
input_ids: [[168, 243, 230, 167, 227, 243, 47991, 246, 168, 226, 116, 168, 248, 242]]
토큰 개수: 14
--------------------------------------------------
텍스트: 'Hello world'
토큰: ['Hello', 'Ġworld']
input_ids: [[15496, 995]]
토큰 개수: 2
--------------------------------------------------
텍스트: '프로그래밍'
토큰: ['í', 'Ķ', 'Ħ', 'ë', '¡', 'ľ', 'ê', '·', '¸', 'ë', 'ŀ', 'ĺ', 'ë', '°', 'į']
input_ids: [[169, 242, 226, 167, 94, 250, 166, 115, 116, 167, 252, 246, 167, 108, 235]]
토큰 개수: 15
--------------------------------------------------
텍스트: 'AI는 미래다'
토큰: ['AI', 'ë', 'Ĭ', 'Ķ', 'Ġë', '¯', '¸', 'ë', 'ŀ', 'ĺ', 'ëĭ', '¤']
input_ids: [[20185, 167, 232, 242, 31619, 107, 116, 167, 252, 246, 46695, 97]]
토큰 개수: 12
--------------------------------------------------
텍스트: '123456'
토큰: ['123', '456']
input_ids: [[10163, 29228]]
토큰 개수: 2
--------------------------------------------------
텍스트: 'hello@email.com

## **예제 3: 토큰 ID의 사용 과정**

In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# 모델과 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 원본 텍스트
input_text = "프로그래밍은 재미있다. 프로그래밍은"
print(f"입력 텍스트: '{input_text}'")

# 토크나이제이션
input_ids = tokenizer.encode(input_text, return_tensors='pt')
print(f"input_ids: {input_ids}")
print(f"각 ID가 나타내는 토큰:")

# 각 ID가 무슨 토큰인지 확인
for i, token_id in enumerate(input_ids[0]):
    token = tokenizer.decode([token_id])
    print(f"  위치 {i}: ID {token_id.item()} → '{token}'")

print()

# 모델에 입력하여 다음 토큰 확률 계산
with torch.no_grad():
    outputs = model(input_ids)
    # logits: [배치_크기, 시퀀스_길이, 어휘_크기]
    logits = outputs.logits

print(f"모델 출력 형태: {logits.shape}")
print(f"마지막 토큰 위치의 확률 분포 크기: {logits[0, -1, :].shape}")

# 다음 토큰으로 가능성이 높은 상위 5개 확인
last_token_logits = logits[0, -1, :]
probabilities = torch.softmax(last_token_logits, dim=-1)
top_5_prob, top_5_indices = torch.topk(probabilities, 5)

print("\n다음 토큰 예측 상위 5개:")
for i, (prob, idx) in enumerate(zip(top_5_prob, top_5_indices)):
    token = tokenizer.decode([idx])
    print(f"{i+1}. '{token}' (확률: {prob:.4f})")

입력 텍스트: '프로그래밍은 재미있다. 프로그래밍은'
input_ids: tensor([[  169,   242,   226,   167,    94,   250,   166,   115,   116,   167,
           252,   246,   167,   108,   235, 35975,   222, 23821,   252,   105,
           167,   107,   116,   168,   252,   230, 46695,    97,    13,   220,
           169,   242,   226,   167,    94,   250,   166,   115,   116,   167,
           252,   246,   167,   108,   235, 35975,   222]])
각 ID가 나타내는 토큰:
  위치 0: ID 169 → '�'
  위치 1: ID 242 → '�'
  위치 2: ID 226 → '�'
  위치 3: ID 167 → '�'
  위치 4: ID 94 → '�'
  위치 5: ID 250 → '�'
  위치 6: ID 166 → '�'
  위치 7: ID 115 → '�'
  위치 8: ID 116 → '�'
  위치 9: ID 167 → '�'
  위치 10: ID 252 → '�'
  위치 11: ID 246 → '�'
  위치 12: ID 167 → '�'
  위치 13: ID 108 → '�'
  위치 14: ID 235 → '�'
  위치 15: ID 35975 → '�'
  위치 16: ID 222 → '�'
  위치 17: ID 23821 → ' �'
  위치 18: ID 252 → '�'
  위치 19: ID 105 → '�'
  위치 20: ID 167 → '�'
  위치 21: ID 107 → '�'
  위치 22: ID 116 → '�'
  위치 23: ID 168 → '�'
  위치 24: ID 252 → '�'
  위치 25: ID 230 → '�'
  

- **[주의!] 이상한 토큰들의 정체**
    - 출력 결과에서 GPT2는 한글을 바이트 단위로 분해 때문에 토큰의 이상하게 보여질 수 있다⭢의미 손실 발생할 수 있다.
- 의미 단위로 분해해야 정확함
    - ✅ 원본 텍스트: '프로그래밍은 재미있다. 프로그래밍은'
    - 1️⃣ 토큰 분할 결과: ['í', 'Ķ', 'Ħ', 'ë', '¡', 'ľ', 'ê', '·', '¸', 'ë', 'ŀ', 'ĺ', 'ë', '°', 'į', 'ìĿ', 'Ģ', 'Ġì', 'ŀ', '¬', 'ë', '¯', '¸', 'ì', 'ŀ', 'Ī', 'ëĭ', '¤', '.', 'Ġ', 'í', 'Ķ', 'Ħ', 'ë', '¡', 'ľ', 'ê', '·', '¸', 'ë', 'ŀ', 'ĺ', 'ë', '°', 'į', 'ìĿ', 'Ģ']
- ['í', 'Ķ', 'Ħ', 'ë', '¡', 'ľ' ...] 이 토큰들의 정체

|정체|설명|문제점|
|--- |--- |--- |
|UTF-8 바이트의 잘못된 해석 |한글 바이트를 Latin-1로 디코딩한 결과 |의미 완전 손실 |
|BPE 알고리즘의 한계 |영어 위주 학습으로 한글 패턴 미학습 |비효율적 토큰화 |
|어휘집 부족 |GPT-2 어휘집에 한글 토큰 거의 없음 |알 수 없는 토큰으로 처리 |

### **1단계: 문제 상황 정확한 분석**

In [9]:
from transformers import GPT2Tokenizer
import warnings
warnings.filterwarnings('ignore')

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 한글 텍스트
korean_text = "프로그래밍은 재미있다"
print(f"원본 텍스트: '{korean_text}'")
print()

# 토큰 분할 결과
tokens = tokenizer.tokenize(korean_text)
print(f"토큰 개수: {len(tokens)}개")
print(f"토큰들: {tokens[:10]}... (처음 10개만 표시)")
print()

# 이상한 문자들의 정체 확인
print("=== 이상한 토큰들의 정체 ===")
for i, token in enumerate(tokens[:5]):
    # 토큰을 바이트로 변환해보기
    try:
        token_id = tokenizer.convert_tokens_to_ids([token])[0]
        print(f"토큰 {i+1}: '{token}' → ID: {token_id}")
    except:
        print(f"토큰 {i+1}: '{token}' → 변환 불가")

print()

# UTF-8 바이트 분석
print("=== UTF-8 바이트 레벨 분석 ===")
korean_bytes = korean_text.encode('utf-8')
print(f"한글 텍스트의 UTF-8 바이트: {korean_bytes}")
print(f"바이트 개수: {len(korean_bytes)}개")

# 각 바이트를 개별 문자로 디코딩 시도
print("바이트별 분석:")
for i, byte_val in enumerate(korean_bytes[:10]):
    print(f"바이트 {i+1}: {byte_val} (0x{byte_val:02x})")

원본 텍스트: '프로그래밍은 재미있다'

토큰 개수: 28개
토큰들: ['í', 'Ķ', 'Ħ', 'ë', '¡', 'ľ', 'ê', '·', '¸', 'ë']... (처음 10개만 표시)

=== 이상한 토큰들의 정체 ===
토큰 1: 'í' → ID: 169
토큰 2: 'Ķ' → ID: 242
토큰 3: 'Ħ' → ID: 226
토큰 4: 'ë' → ID: 167
토큰 5: '¡' → ID: 94

=== UTF-8 바이트 레벨 분석 ===
한글 텍스트의 UTF-8 바이트: b'\xed\x94\x84\xeb\xa1\x9c\xea\xb7\xb8\xeb\x9e\x98\xeb\xb0\x8d\xec\x9d\x80 \xec\x9e\xac\xeb\xaf\xb8\xec\x9e\x88\xeb\x8b\xa4'
바이트 개수: 31개
바이트별 분석:
바이트 1: 237 (0xed)
바이트 2: 148 (0x94)
바이트 3: 132 (0x84)
바이트 4: 235 (0xeb)
바이트 5: 161 (0xa1)
바이트 6: 156 (0x9c)
바이트 7: 234 (0xea)
바이트 8: 183 (0xb7)
바이트 9: 184 (0xb8)
바이트 10: 235 (0xeb)


### **2단계: 영어와 한글 토크나이제이션 비교**

In [10]:
from transformers import GPT2Tokenizer
import warnings
warnings.filterwarnings('ignore')

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 비교 텍스트들
texts = {
    "영어": "Programming is fun",
    "한글": "프로그래밍은 재미있다",
    "숫자": "12345",
    "특수문자": "Hello! @#$%"
}

print("=== 언어별 토크나이제이션 비교 ===")
for lang, text in texts.items():
    tokens = tokenizer.tokenize(text)
    token_count = len(tokens)
    char_count = len(text)

    print(f"\n{lang}: '{text}'")
    print(f"  문자 수: {char_count}")
    print(f"  토큰 수: {token_count}")
    print(f"  효율성: {token_count/char_count:.2f} (토큰/문자)")
    print(f"  토큰 예시: {tokens[:5]}...")

=== 언어별 토크나이제이션 비교 ===

영어: 'Programming is fun'
  문자 수: 18
  토큰 수: 4
  효율성: 0.22 (토큰/문자)
  토큰 예시: ['Program', 'ming', 'Ġis', 'Ġfun']...

한글: '프로그래밍은 재미있다'
  문자 수: 11
  토큰 수: 28
  효율성: 2.55 (토큰/문자)
  토큰 예시: ['í', 'Ķ', 'Ħ', 'ë', '¡']...

숫자: '12345'
  문자 수: 5
  토큰 수: 2
  효율성: 0.40 (토큰/문자)
  토큰 예시: ['123', '45']...

특수문자: 'Hello! @#$%'
  문자 수: 11
  토큰 수: 5
  효율성: 0.45 (토큰/문자)
  토큰 예시: ['Hello', '!', 'Ġ@', '#$', '%']...


### **3단계: 올바른 다국어 토크나이저 사용**

In [11]:
from transformers import AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

# 다국어 지원 토크나이저들 비교
tokenizer_models = {
    "GPT-2 (영어 전용)": "gpt2",
    "mBERT (다국어)": "bert-base-multilingual-cased",
    "XLM-RoBERTa (다국어)": "xlm-roberta-base",
    "KoBERT (한국어)" : "skt/kobert-base-v1",
    "KoGPT2 Base v2 (한국어)" : "skt/kogpt2-base-v2",   # GPT-2 기반
    "Ko-GPT-Trinity 1.2B (v0.5) (한국어)" : "skt/ko-gpt-trinity-1.2B-v0.5"  # GPT-3 스타일
}

korean_text = "프로그래밍은 재미있다"

print("=== 다양한 토크나이저 비교 ===")
for model_name, model_id in tokenizer_models.items():
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        tokens = tokenizer.tokenize(korean_text)
        token_ids = tokenizer.convert_tokens_to_ids(tokens)

        print(f"\n{model_name}:")
        print(f"  토큰 수: {len(tokens)}")
        print(f"  토큰들: {tokens}")
        print(f"  토큰ID: {token_ids}")

        # 역변환 확인
        reconstructed = tokenizer.convert_tokens_to_string(tokens)
        print(f"  역변환: '{reconstructed}'")
        print(f"  원본과 동일: {'✅' if reconstructed.strip() == korean_text else '❌'}")

    except Exception as e:
        print(f"\n{model_name}: 로드 실패 - {e}")

=== 다양한 토크나이저 비교 ===

GPT-2 (영어 전용):
  토큰 수: 28
  토큰들: ['í', 'Ķ', 'Ħ', 'ë', '¡', 'ľ', 'ê', '·', '¸', 'ë', 'ŀ', 'ĺ', 'ë', '°', 'į', 'ìĿ', 'Ģ', 'Ġì', 'ŀ', '¬', 'ë', '¯', '¸', 'ì', 'ŀ', 'Ī', 'ëĭ', '¤']
  토큰ID: [169, 242, 226, 167, 94, 250, 166, 115, 116, 167, 252, 246, 167, 108, 235, 35975, 222, 23821, 252, 105, 167, 107, 116, 168, 252, 230, 46695, 97]
  역변환: '프로그래밍은 재미있다'
  원본과 동일: ✅

mBERT (다국어):
  토큰 수: 8
  토큰들: ['프로', '##그', '##래', '##밍', '##은', '재', '##미', '##있다']
  토큰ID: [102574, 78136, 37388, 118960, 10892, 9659, 22458, 76820]
  역변환: '프로그래밍은 재미있다'
  원본과 동일: ✅

XLM-RoBERTa (다국어):
  토큰 수: 6
  토큰들: ['▁프로', '그래', '밍', '은', '▁재미있', '다']
  토큰ID: [54099, 61882, 144667, 697, 157403, 1875]
  역변환: '프로그래밍은 재미있다'
  원본과 동일: ✅

KoBERT (한국어):
  토큰 수: 14
  토큰들: ['▁', '프로', 'ᄀ', 'ᅳ래ᄆ', 'ᅵ', 'ᆼ', 'ᄋ', 'ᅳᆫ', '▁', '재ᄆ', 'ᅵ', 'ᄋ', 'ᅵ', 'ᆻ다']
  토큰ID: [517, 0, 490, 0, 494, 0, 491, 0, 517, 0, 494, 491, 494, 0]
  역변환: '프로그래밍은 재미있다'
  원본과 동일: ❌

KoGPT2 Base v2 (한국어):
  토큰 수: 7
  토큰들: ['▁프로

- 한글 처리를 위한 올바른 선택

|용도 |권장 모델 |이유
|--- |--- |--- |
| 한글 텍스트 생성| GPT-3.5/4, KoGPT| 한글 데이터로 훈련됨|
| 한글 이해/분류| KoBERT, KoELECTRA| 한국어 특화 모델|
| 다국어 처리| mBERT, XLM-RoBERTa| 다국어 동시 지원|
| 실습/학습용| 영어 예제 사용| GPT-2 본래 성능 확인|

- 학습용 개선된 예제

In [12]:
# 올바른 접근: 영어로 실습하기
from transformers import GPT2Tokenizer
import warnings
warnings.filterwarnings('ignore')

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# GPT-2가 잘 처리하는 영어 텍스트
english_text = "Programming is fun. Programming is"
print(f"영어 텍스트: '{english_text}'")

tokens = tokenizer.tokenize(english_text)
print(f"토큰들: {tokens}")
print(f"토큰 수: {len(tokens)}")

# 각 토큰의 의미 확인
print("\n토큰별 의미:")
for i, token in enumerate(tokens):
    print(f"{i+1}. '{token}' → 의미있는 단위")

영어 텍스트: 'Programming is fun. Programming is'
토큰들: ['Program', 'ming', 'Ġis', 'Ġfun', '.', 'ĠProgramming', 'Ġis']
토큰 수: 7

토큰별 의미:
1. 'Program' → 의미있는 단위
2. 'ming' → 의미있는 단위
3. 'Ġis' → 의미있는 단위
4. 'Ġfun' → 의미있는 단위
5. '.' → 의미있는 단위
6. 'ĠProgramming' → 의미있는 단위
7. 'Ġis' → 의미있는 단위




---

